In [1]:
import numpy as np

In [2]:
np.random.seed(21)

In [3]:
from collections import defaultdict

In [4]:
import scipy.stats as st
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, UpSampling2D, AveragePooling2D, Flatten, Dense, Lambda
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.losses import mean_squared_error, mean_absolute_error, categorical_crossentropy, binary_crossentropy
from keras.preprocessing.image import Iterator
import matplotlib.pyplot as plt
import cv2
import os
import json
import pickle
%matplotlib inline

from keras.utils.np_utils import to_categorical

from keras.layers import Reshape, BatchNormalization

import tensorflow as tf
sess = tf.Session()
K.set_session(sess)

Using TensorFlow backend.


In [5]:
from __future__ import absolute_import
from __future__ import print_function

from pkg_resources import parse_version
from keras.callbacks import Callback

In [ ]:
class TensorBoardCallBack(Callback):
    """Tensorboard basic visualizations.
    This callback writes a log for TensorBoard, which allows
    you to visualize dynamic graphs of your training and test
    metrics, as well as activation histograms for the different
    layers in your model.
    TensorBoard is a visualization tool provided with TensorFlow.
    If you have installed TensorFlow with pip, you should be able
    to launch TensorBoard from the command line:
    ```
    tensorboard --logdir=/full_path_to_your_logs
    ```
    You can find more information about TensorBoard
    [here](https://www.tensorflow.org/versions/master/how_tos/summaries_and_tensorboard/index.html).
    # Arguments
        log_dir: the path of the directory where to save the log
            files to be parsed by Tensorboard
        batch_freq: frequency (in batch) at which to log data
            If set to 0, we just log at the end of an epoch,
    """

    def __init__(self, log_dir='./logs',
                 batch_freq=0):
        super(TensorBoardCallBack, self).__init__()

        self.log_dir = log_dir
        self.batch_freq = batch_freq
        self.merged = None
        self.writer = tf.summary.FileWriter(self.log_dir)
        self.last_batch = 0
        self.batch_offset = 0

    def set_model(self, model):
        self.model = model
        self.sess = K.get_session()

        if hasattr(tf, 'merge_all_summaries'):
            self.merged = tf.merge_all_summaries()
        else:
            self.merged = tf.summary.merge_all()

    def on_batch_end(self, batch, logs=None):
        # Each time we go back to batch 0, we increase the batch_offset
        if batch < self.last_batch:
            self.batch_offset += self.last_batch + 1
        self.last_batch = batch

        batch_cross_epoch = self.batch_offset + batch
        if batch_cross_epoch % self.batch_freq == 0:
            logs = logs or {}

            for name, value in logs.items():
                if name in ['batch', 'size']:
                    continue
                summary = tf.Summary()
                summary_value = summary.value.add()
                summary_value.simple_value = value.item()
                summary_value.tag = name
                self.writer.add_summary(summary, batch_cross_epoch)
            self.writer.flush()

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        for name, value in logs.items():
            if name in ['batch', 'size']:
                continue
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.writer.add_summary(summary, epoch)
        self.writer.flush()

    def on_train_end(self, _):
        self.writer.close()

In [ ]:
from keras.applications.resnet50 import ResNet50

In [ ]:
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(None, None, 3))

In [ ]:
h = base_model.layers[-2]
sealion_prediction = Conv2D(6, (3, 3), activation="softmax", name='predictions')(h.output)

In [ ]:
sealion_net = Model(inputs=[base_model.input], outputs=[sealion_prediction])

In [ ]:
sealion_net.predict(np.ones((1, 69, 69, 3))).shape

In [ ]:
sealion_net.summary()

In [ ]:
class NonValidPatch(Exception):
    pass

In [ ]:
def get_block_loc(shape, x, y, target_size=(224, 224), n_blocks=(4,4), overlap=(448,448)):
    h, w = shape
    w_block = (w + (n_blocks[1] - 1) * overlap[1]) // n_blocks[1]
    h_block = (h + (n_blocks[0] - 1) * overlap[0]) // n_blocks[0]
    for by in range(n_blocks[0]):
        y_start = by * (h_block - overlap[0])
        y_end = y_start + h_block + 1
        for bx in range(n_blocks[1]):
            x_start = bx * (w_block - overlap[1])
            x_end = x_start + w_block + 1
            
            if x_start <= x < x_end and y_start <= y < y_end and\
            x_start <= x + target_size[1] - 1 < x_end and y_start <= y + target_size[0] - 1 < y_end:
                return bx + by * n_blocks[0], x - x_start, y - y_start
    raise NonValidPatch("Can't find block...??")

In [ ]:
class PatchesIterator(Iterator):
    """Iterator yielding training samples of subadult_males
    :param root_dir: Directory containing training images, and dots.
    :param image_ids: Set of image ids to use to sample patches.
    :param class_weights: Weights for each class.
    :param n_samples_per_image: Number of patches to sample on each image.
    :param target_size: Size of the patches sampled.
    :param batch_size: Number of patches sampled per batch
    :param shuffle: Boolean, whether to shuffle the data between epochs.
    :param seed: Random seed for data shuffling.
    :return batch_x, batch_x. 
        batch_x is a (batch_size, target_size[0], target_size[1], 3) array
        batch_x is a (batch_size, target_size[0], target_size[1], 1) array if output_counts is False
        otherwise, it is a (batch_size, 5) array.
    """

    def __init__(self, root_dir, image_ids,
                 class_weights = None,
                 n_samples_per_image=160,
                 target_size=(69, 69),
                 batch_size=64, shuffle=True, seed=42, debug_dir=None):
        self.n_samples_per_block = 16
        self.n_sealion_types = 5
        self.image_ids = image_ids
        self.root_dir = root_dir
        self.debug_dir = debug_dir
        # Normalize to use class_weights as a probability distribution.
        if class_weights:
            self.class_weights = np.asarray(class_weights)/np.sum(class_weights)
        else:
            self.class_weights = np.ones((self.n_sealion_types+1))/(self.n_sealion_types + 1)
            
        self.n_samples_per_image = n_samples_per_image
        self.target_size = target_size
        self.n_indices = len(self.image_ids) * self.n_samples_per_image
                 
        super(PatchesIterator, self).__init__(self.n_indices, batch_size//self.n_samples_per_block, shuffle, seed)
        
    def get_class_distribution(self, n_batches=100):
        counts = defaultdict(int)
        for _ in range(n_batches):
            _, by = self.next()
            by = np.argmax(by, axis=-1)
            cls, cnts = np.unique(by.ravel(), return_counts=True)
            for c, cnt in zip(cls, cnts):
                counts[c] += cnt
        return counts
        
    def normalize_input(self, x_bgr):
        x = x_bgr.copy()
        x[..., 0] -= 103.939
        x[..., 1] -= 116.779
        x[..., 2] -= 123.68
        return x
    
    def denormalize_input(self, x_normed):
        x = x_normed.copy()
        x[..., 0] += 103.939
        x[..., 1] += 116.779
        x[..., 2] += 123.68
        return x

    def random_transform(self, x):
        flips = np.random.randint(0, 2, (3,))
        if flips[0]:
            x = np.rot90(x)
        if flips[1]:
            x = np.flipud(x)
        if flips[2]:
            x = np.fliplr(x)
        return x
    
    def get_weights(self, dots):
        # Set probability to 0 if some sealion type is not in the block
        current_weigths = self.class_weights.copy()
        for i in range(self.n_sealion_types):
            if not dots[i]:
                current_weigths[i] = 0
        current_weigths /= np.sum(current_weigths)
        return current_weigths
    
    def sample_position(self, shape, dots, current_weigths, size):
        # Choose an output class randomly
        output_class = np.random.choice(self.n_sealion_types + 1, size=(1, ), p=current_weigths)[0]
        # Sample a location, either for background or for a sealion.
        if output_class == self.n_sealion_types:
            # avoid bg with sealions in it
            return self.sample_bg(shape, dots, size), output_class
        else:
            return self.sample_dot(shape, dots[output_class], size), output_class

    def get_dots_in_block(self, bid, shape, dots, n_blocks=(4,4), overlap=(448,448)):
        h, w = shape
        w_block = (w + (n_blocks[1] - 1) * overlap[1]) // n_blocks[1]
        h_block = (h + (n_blocks[0] - 1) * overlap[0]) // n_blocks[0]
        
        bx = bid % n_blocks[0]
        by = bid // n_blocks[0]
        
        y_start = by * (h_block - overlap[0])
        y_end = y_start + h_block + 1
        x_start = bx * (w_block - overlap[1])
        x_end = x_start + w_block + 1
        
        dots_in_block = [[] for _ in range(self.n_sealion_types)]
        for i, ds in enumerate(dots):
            for (x, y) in ds:
                if x_start <= x < x_end and y_start <= y < y_end:
                    dots_in_block[i].append((x - x_start, y - y_start))
        return dots_in_block
        
    def get_random_block(self, image_id, shape, dots, current_weigths):
        while True:
            try:
                (x, y), output_class = self.sample_position(shape, dots, current_weigths, self.target_size)
                
                # Get the corresponding image block, and (x, y) in this block
                bid, x, y = get_block_loc(shape, x, y)
                
                # Load the block and check if it is valid
                uid = "{iid}_{bid}".format(iid=image_id, bid=bid)
                img = cv2.imread(os.path.join(self.root_dir, "TrainBlock", uid + ".jpg"))
                if img is not None:
                    return bid, img
            except NonValidPatch:
                continue
        
    
    def sample(self, shape, dots, image_id):   
        # If we are stuck on a "bad" block, we retry from scratch
        while True:
            # Set probability to 0 if some sealion type is not in the block
            current_weigths = self.get_weights(dots)
        
            # Get a block randomly (but using the dots and the sampling weights)
            bid, img = self.get_random_block(image_id, shape, dots, current_weigths)
                
            # Recompute dots in the blocks, and sampling weights
            dots_block = self.get_dots_in_block(bid, shape, dots)
            current_weigths = self.get_weights(dots_block)

            # Now, sample n_samples_per_block patches from it
            n_samples = 0
            bx = np.zeros((self.n_samples_per_block, self.target_size[0], self.target_size[1], 3))
            by = np.zeros((self.n_samples_per_block, ))

            # Stop if we try too many times, 
            max_iterations = self.n_samples_per_block * 5
            current_iteration = 0
            while n_samples < self.n_samples_per_block and current_iteration < max_iterations:
                current_iteration += 1
                try:
                    h, w = self.target_size                
                    (x, y), output_class = self.sample_position(img.shape[:2], dots_block, current_weigths, (h, w))
                    img_patch = img[y:y+h, x:x+w,:]

                    if img_patch.shape[0] != self.target_size[0] or img_patch.shape[1] != self.target_size[1]:
                        continue
                    
                    bx[n_samples, ...] = self.random_transform(img_patch)
                    by[n_samples] = output_class
                    n_samples += 1
                except NonValidPatch:
                    continue
                    
            if current_iteration < max_iterations:
                return bx, by
            # else, we tried too many times, let's get another block.
        
        
    def contains_dots(self, xstart, ystart, dots, margin):
        x1 = xstart - margin
        y1 = ystart - margin
        x2 = xstart + self.target_size[1] + margin
        y2 = ystart + self.target_size[0] + margin
        for ds in dots:
            for (x, y) in ds:
                if x1 <= x < x2 and y1 <= y < y2:
                    return True
        return False
    
    def sample_bg(self, shape, dots, size):
        margin = 50 # more than half of the biggest sealion expected size
        max_iterations = 10
        current_iteration = 0
        while current_iteration < max_iterations:
            if shape[1] - size[1] <= 0 or shape[0] - size[0] <= 0:
                raise NonValidPatch("Cant' find background")
            x = np.random.randint(0, shape[1] - size[1], size=(1,))[0]
            y = np.random.randint(0, shape[0] - size[0], size=(1,))[0]
            if not self.contains_dots(x, y, dots, margin):
                return x, y
            current_iteration += 1
        raise NonValidPatch("Cant' find background")
    
    def sample_dot(self, shape, dots, size):
        margin = 6  # dot will can be in [-6, 6] from the center of the patch
        
        rand_index = np.random.choice(len(dots), size=(1,))[0]
        rand_dot = dots[rand_index]
        
        min_x = max(0, rand_dot[0] - size[1]//2 - margin)
        max_x = max(0, min(shape[1] - size[1], rand_dot[0] - size[1]//2 + margin))
        
        min_y = max(0, rand_dot[1] - size[0]//2 - margin)
        max_y = max(0, min(shape[0] - size[0], rand_dot[1] - size[0]//2 + margin))
        
        if min_x > max_x:
            max_x, min_x = min_x, max_x
        if min_y > max_y:
            max_y, min_y = min_y, max_y 
            
        if min_x == max_x or min_y == max_y:
            raise NonValidPatch()
           
        x = np.random.randint(min_x, max_x, size=(1,))[0]
        y = np.random.randint(min_y, max_y, size=(1,))[0]
        
        return x, y
        
    def next(self):
        """For python 2.x.
        # Returns
            The next batch.
        """
        # Keeps under lock only the mechanism which advances
        # the indexing of each batch.
        with self.lock:
            index_array, current_index, current_batch_size = next(self.index_generator)
                
        batch_x = np.zeros((current_batch_size * self.n_samples_per_block, self.target_size[0], self.target_size[1], 3), dtype=K.floatx())
        batch_y = np.zeros((current_batch_size * self.n_samples_per_block), dtype=np.int32)
        
        # For each index, we load the data and sample randomly n_successive_samples patches
        for i, j in enumerate(index_array):
            index = j // self.n_samples_per_image
            image_id = self.image_ids[index]
            with open(os.path.join(self.root_dir, "TrainDots", str(image_id) + ".pkl"), "rb") as pfile:
                dots = pickle.load(pfile)
            with open(os.path.join(self.root_dir, "TrainShape", str(image_id) + ".pkl"), "rb") as pfile:
                shape = pickle.load(pfile)
                
            x, y = self.sample(shape, dots, image_id)
            batch_x[i*self.n_samples_per_block:(i+1)*self.n_samples_per_block, ...] = x
            batch_y[i*self.n_samples_per_block:(i+1)*self.n_samples_per_block] = y 

        if self.debug_dir:
            for i in range(current_batch_size):
                cv2.imwrite(os.path.join(self.debug_dir, "patch_{}.jpg".format(i)), batch_x[i])
                
        permut = np.random.permutation(batch_x.shape[0])
        return self.normalize_input(batch_x[permut, ...]), to_categorical(batch_y[permut,...].ravel(), num_classes=6).reshape((-1, 1, 1, 6))

In [ ]:
with open("../data/train.json", "r") as jfile:
    train_ids = json.load(jfile)
train_ids = [int(iid) for iid in train_ids]

with open("../data/val.json", "r") as jfile:
    val_ids = json.load(jfile)
val_ids = [int(iid) for iid in val_ids]

In [ ]:
class_weights = [0.2, 0.2, 0.2, 0.2, 0.2, 0.1]

In [ ]:
trainNoPupsIterator = PatchesIterator("/home/ubuntu/sealion/data/", train_ids, class_weights=class_weights)

valNoPupsIterator = PatchesIterator("/home/ubuntu/sealion/data/", val_ids, class_weights=class_weights)

In [ ]:
trainNoPupsIterator.get_class_distribution(100)

In [ ]:
for i, (batch_x, batch_y) in enumerate(trainNoPupsIterator):
    break

In [ ]:
plt.imshow(trainNoPupsIterator.denormalize_input(batch_x[61, ...]))

In [ ]:
np.argmax(batch_y, axis=-1).ravel()

In [ ]:
sealion_net.predict(batch_x).shape

In [ ]:
np.sum(sealion_net.predict(batch_x)[0,...]) #check softmax

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
from keras.losses import categorical_crossentropy

In [ ]:
sgd = SGD(lr=5*1e-4, momentum=0.9, decay=1e-6, nesterov=True)
sealion_net.compile(optimizer=sgd, loss=categorical_crossentropy, metrics=['accuracy'])

In [ ]:
!rm -rf /home/ubuntu/data/sealion/data/models/classification_all/*

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau

cb_checkpoint = ModelCheckpoint("/home/ubuntu/data/sealion/data/models/classification_all/ckpt_{epoch:02d}-{val_loss:.2f}.h5")
cb_reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, mode='auto', epsilon=0.01, cooldown=0, min_lr=0)
tensorboard_cb = TensorBoardCallBack(log_dir="/home/ubuntu/data/sealion/data/models/classification_all/log_tb", batch_freq=10)

In [ ]:
h = sealion_net.fit_generator(trainNoPupsIterator, 1000, epochs=20,
                              verbose=1, callbacks=[cb_checkpoint, cb_reduce_lr, tensorboard_cb],
                              validation_data=valNoPupsIterator, validation_steps=200,
                              class_weight=None,
                              max_q_size=16, workers=4, pickle_safe=False,
                              initial_epoch=0)

In [ ]:
for i, (batch_x, batch_y) in enumerate(valNoPupsIterator):
    break

In [ ]:
batch_ypred = sealion_net.predict(batch_x)

In [ ]:
np.argmax(batch_y, axis=-1).ravel()

In [ ]:
np.argmax(batch_ypred, axis=-1).ravel()

In [ ]:
plt.imshow(trainNoPupsIterator.denormalize_input(batch_x[-4, ...]))

## Check on a bigger input

In [ ]:
im_val = cv2.imread("/home/ubuntu/sealion/data/TrainBlock/43_5.jpg")

plt.imshow(im_val)

In [ ]:
im_val.shape

In [ ]:
im_val_patch = im_val[:837, -837:, :]
im_val_patch = im_val_patch.reshape((1, 837, 837, 3))

In [ ]:
plt.imshow(im_val_patch[0,...])

In [ ]:
im_val_pred = sealion_net.predict(trainNoPupsIterator.normalize_input(im_val_patch.astype(np.float32)))

In [ ]:
im_val_pred.shape

In [ ]:
plt.imshow(np.argmax(im_val_pred[0,...], axis=-1))

In [ ]:
# Unfreeze the last branch
for l in base_model.layers[141:]:
    l.trainable = True

In [ ]:
sgd = SGD(lr=1e-6, momentum=0.9, decay=1e-6, nesterov=True)
sealion_net.compile(optimizer=sgd, loss=categorical_crossentropy, metrics=['accuracy'])

In [ ]:
cb_checkpoint_ft = ModelCheckpoint("/home/ubuntu/data/sealion/data/models/classification_all/ckpt_ft_{epoch:02d}-{val_loss:.2f}.h5")
cb_reduce_lr_ft = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, mode='auto', epsilon=0.001, cooldown=0, min_lr=0)
tensorboard_cb_ft = TensorBoardCallBack(log_dir="/home/ubuntu/data/sealion/data/models/classification_all/log_tb_ft", batch_freq=10)

In [ ]:
h_finetune = sealion_net.fit_generator(trainNoPupsIterator, 1000, epochs=5,
                              verbose=1, callbacks=[cb_checkpoint_ft, cb_reduce_lr_ft, tensorboard_cb_ft],
                              validation_data=valNoPupsIterator, validation_steps=200,
                              class_weight=None,
                              max_q_size=16, workers=4, pickle_safe=False,
                              initial_epoch=0)

## Check again on large patch

In [ ]:
im_val_pred = sealion_net.predict(im_val_patch)

In [ ]:
plt.imshow(im_val_patch[0,...])

In [ ]:
plt.imshow(np.argmax(im_val_pred[0,...], axis=-1))

In [ ]:
count_input_size = 965

In [ ]:
t = np.ones((1, 965, 965, 3))

In [ ]:
sealion_net.predict(t).shape

# Create a CNN to get the count

In [ ]:
from keras.models import load_model

In [ ]:
sealion_net = load_model("/home/ubuntu/data/sealion/data/models/classification_all/ckpt_ft_04-1.04.h5")

In [ ]:
x = sealion_net.output
x = BatchNormalization(name='bn_heatmap')(x)
x = Conv2D(64, (29, 29), activation='relu', name='cnt_d1')(x)
x = BatchNormalization(name='bn_cnt_d1')(x)
x = Conv2D(32, (1, 1), activation='relu', name='cnt_d2')(x)
counts = Conv2D(5, (1, 1), activation='relu', name='counts')(x)

In [ ]:
sealion_count_net = Model(inputs=[sealion_net.input], outputs=[counts])

In [ ]:
sealion_count_net.predict(np.ones((1, count_input_size, count_input_size, 3))).shape

In [ ]:
sealion_count_net.summary()

In [ ]:
class CountsIterator(Iterator):
    def __init__(self, root_dir, image_ids,
                 n_samples_per_image=160,
                 target_size=(965, 965),
                 batch_size=16, shuffle=True, seed=42, debug_dir=None):
        
        self.n_sealion_types = 5
        self.image_ids = image_ids
        self.root_dir = root_dir
        self.debug_dir = debug_dir
        self.n_samples_per_block = 8
        self.n_samples_per_image = n_samples_per_image
        self.target_size = target_size
        self.n_indices = len(self.image_ids) * self.n_samples_per_image
                 
        super(CountsIterator, self).__init__(self.n_indices, batch_size//self.n_samples_per_block, shuffle, seed)
    
    def normalize_input(self, x_bgr):
        x = x_bgr.copy()
        x[..., 0] -= 103.939
        x[..., 1] -= 116.779
        x[..., 2] -= 123.68
        return x
    
    def denormalize_input(self, x_normed):
        x = x_normed.copy()
        x[..., 0] += 103.939
        x[..., 1] += 116.779
        x[..., 2] += 123.68
        return x

    def random_transform(self, im):
        flips = np.random.randint(0, 2, (3,))
        if flips[0]:
            x = np.rot90(x)
            y = np.rot90(y)
        if flips[1]:
            x = np.flipud(x)
            y = np.flipud(y)
        if flips[2]:
            x = np.fliplr(x)
            y = np.fliplr(y)
        return x, y
    
    def sample(self, im, dots):
        h, w, c = im.shape
        batch_x = np.zeros((self.n_samples_per_block, self.target_size[0], self.target_size[1], 3), dtype=np.float32)
        batch_y = np.zeros((self.n_samples_per_block, 5), dtype=np.float32)
        xs = np.random.randint(0, w - self.target_size[1], size=(self.n_samples_per_block,))
        ys = np.random.randint(0, h - self.target_size[0], size=(self.n_samples_per_block,))
        for i in range(self.n_samples_per_block):
            counts = self.get_counts(xs[i], ys[i], dots)
            batch_x[i, ...] = im[ys[i]:ys[i]+self.target_size[0], xs[i]:xs[i]+self.target_size[1],...]
            batch_y[i, ...] = np.asarray(counts, dtype=np.float32)
        return batch_x, batch_y
    
    def get_counts(self, xstart, ystart, dots):
        x1 = xstart
        y1 = ystart
        x2 = xstart + self.target_size[1]
        y2 = ystart + self.target_size[0]
        counts = [0, 0, 0, 0, 0]
        for i, ds in enumerate(dots):
            for (x, y) in ds:
                if x1 <= x < x2 and y1 <= y < y2:
                    counts[i] += 1
        return counts
        
    def next(self):
        """For python 2.x.
        # Returns
            The next batch.
        """
        # Keeps under lock only the mechanism which advances
        # the indexing of each batch.
        with self.lock:
            index_array, current_index, current_batch_size = next(self.index_generator)
                
        batch_x = np.zeros((current_batch_size * self.n_samples_per_block,
                            self.target_size[0],
                            self.target_size[1],
                            3),
                           dtype=K.floatx())
        batch_y = np.zeros((current_batch_size * self.n_samples_per_block, 5),
                           dtype=np.int32)
        
        # For each index, we load the data and sample randomly n_successive_samples patches
        for i, j in enumerate(index_array):
            index = j // self.n_samples_per_image
            image_id = self.image_ids[index]
            with open(os.path.join(self.root_dir, "TrainDots", str(image_id) + ".pkl"), "rb") as pfile:
                dots = pickle.load(pfile)
            im = cv2.imread(os.path.join(self.root_dir, "Train", str(image_id) + ".jpg"))
                
            x, y = self.sample(im, dots)
            batch_x[i*self.n_samples_per_block:(i+1)*self.n_samples_per_block, ...] = x
            batch_y[i*self.n_samples_per_block:(i+1)*self.n_samples_per_block, ...] = y 

        if self.debug_dir:
            for i in range(batch_x.shape[0]):
                cv2.imwrite(os.path.join(self.debug_dir, "patch_{}.jpg".format(i)), batch_x[i])
        
        return self.normalize_input(batch_x), batch_y.reshape(-1, 1, 1, 5)

In [ ]:
trainCountsGenerator = CountsIterator("/home/ubuntu/sealion/data/", train_ids)
valCountsGenerator = CountsIterator("/home/ubuntu/sealion/data/", val_ids)

In [ ]:
for bx, by in trainCountsGenerator:
    break

In [ ]:
bx.shape

In [ ]:
plt.imshow(trainCountsGenerator.denormalize_input(bx)[0])

In [ ]:
by.shape

In [ ]:
sealion_count_net.predict(bx).shape

In [ ]:
for l in sealion_count_net.layers:
    l.trainable = False

In [ ]:
sealion_count_net.summary()

In [ ]:
for l in sealion_count_net.layers[-5:]:
    l.trainable = True

In [ ]:
sealion_count_net.summary()

In [ ]:
sgd = SGD(lr=1e-4, momentum=0.9, decay=1e-6, nesterov=True)
sealion_count_net.compile(optimizer=sgd, loss=mean_squared_error)

In [ ]:
cb_checkpoint_cnt = ModelCheckpoint("/home/ubuntu/data/sealion/data/models/classification_all/ckpt_cnt_{epoch:02d}-{val_loss:.2f}.h5")
cb_reduce_lr_cnt = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, mode='auto', epsilon=1.0, cooldown=0, min_lr=0)
tensorboard_cb_cnt = TensorBoardCallBack(log_dir="/home/ubuntu/data/sealion/data/models/classification_all/log_tb_cnt", batch_freq=10)

In [ ]:
h_cnt = sealion_count_net.fit_generator(trainCountsGenerator, 1000, epochs=5,
                           verbose=1, callbacks=[cb_checkpoint_cnt, cb_reduce_lr_cnt, tensorboard_cb_cnt],
                           validation_data=valCountsGenerator, validation_steps=100,
                           class_weight=None,
                           max_q_size=4, workers=4, pickle_safe=False,
                           initial_epoch=0)

In [ ]:
sealion_count_net = load_model("/home/ubuntu/data/sealion/data/models/classification_all/ckpt_cnt_00-16.62.h5")

In [ ]:
for l in sealion_count_net.layers[-5 -1 -141:]:
    l.trainable = True

In [ ]:
sgd = SGD(lr=1e-4, momentum=0.9, decay=1e-6, nesterov=True)
sealion_count_net.compile(optimizer=sgd, loss=mean_squared_error)

In [ ]:
h_cnt = sealion_count_net.fit_generator(trainCountsGenerator, 1000, epochs=5,
                           verbose=1, callbacks=[cb_checkpoint_cnt, cb_reduce_lr_cnt, tensorboard_cb_cnt],
                           validation_data=valCountsGenerator, validation_steps=100,
                           class_weight=None,
                           max_q_size=4, workers=4, pickle_safe=False,
                           initial_epoch=0)

In [ ]:
for bx, by in valNoPupsCountsIterator:
    break

In [ ]:
bypred = sealion_count_net.predict(bx)

In [ ]:
by[5,...]

In [ ]:
bypred[5,...]

In [ ]:
def mse(y_true, y_pred):
    return np.mean(np.square(y_pred - y_true), axis=-1)

In [ ]:
np.mean(mse(by, bypred))

In [ ]:
for l in sub_adults_count_net.layers:
    l.Trainable = True

In [ ]:
sgd = SGD(lr=0.000001, momentum=0.9, decay=1e-6, nesterov=True)
sub_adults_count_net.compile(optimizer=sgd, loss=mean_squared_error)

In [ ]:
h = sub_adults_count_net.fit_generator(trainSubMalesCountsIterator, 1000, epochs=5,
                           verbose=1, callbacks=[cb_checkpoint_cnt, cb_reduce_lr_cnt, tensorboard_cb_cnt],
                           validation_data=valSubMalesCountsIterator, validation_steps=100,
                           class_weight=None,
                           max_q_size=8, workers=4, pickle_safe=False,
                           initial_epoch=0)

In [ ]:
1+1

In [ ]:
im_val = cv2.imread("/home/ubuntu/sealion/data/TrainBlock/881_13.jpg")
dmap = np.load("/home/ubuntu/sealion/data/TrainEllipseDensity/881_13.npz")["dmap"]

In [ ]:
plt.imshow(im_val)

In [ ]:
im_val_patch = im_val[0:count_size:, 700:700+count_size, :]
im_val_patch = im_val_patch.reshape((1, count_size, count_size, 3))
dmap_patch = dmap[0:0+count_size:, 700:700+count_size,...]
plt.imshow(im_val_patch[0, ...])

In [ ]:
plt.imshow(dmap_patch[...,4])

In [ ]:
sealion_count_net.predict(trainNoPupsCountsIterator.normalize_input(im_val_patch.astype(np.float32)))

In [ ]:
np.sum(np.sum(dmap_patch, axis=0), axis=0)

In [ ]:
im_val_patch = im_val[400:400+count_size, :count_size, :]
im_val_patch = im_val_patch.reshape((1, count_size, count_size, 3))
dmap_patch = dmap[400:400+count_size, :count_size]
plt.imshow(im_val_patch[0, ...])

In [ ]:
plt.imshow(dmap_patch)

In [ ]:
sealion_count_net.predict(trainNoPupsCountsIterator.normalize_input(im_val_patch.astype(np.float32)))

In [ ]:
np.sum(np.sum(dmap_patch, axis=0), axis=0)

In [ ]:
def predict_counts(im, net, patchGenerator, zeros, patch_size=(count_size, count_size), batch_size=8, overlap=(16, 16)):
    h, w, c = im.shape
    n_patches_x = int(np.ceil((w - patch_size[1])/(patch_size[1] - overlap[1]) + 1))
    n_patches_y = int(np.ceil((h - patch_size[0])/(patch_size[0] - overlap[0]) + 1))
    print(n_patches_x, n_patches_x)
    
    counts = [[] for _ in range(4)]
    batch_pos = []
    patch_count = np.zeros((h, w, 1), dtype=np.int8)
    batch_x = np.zeros((batch_size, ) + patch_size + (c, ), dtype=np.float32)
    
    current_batch_size = 0
    for py in range(n_patches_y):
        y_start = py * (patch_size[0] - overlap[0])
        y_start = min(h - patch_size[0], y_start)
        y_end = y_start + patch_size[0]
        for px in range(n_patches_x):
            x_start = px * (patch_size[1] - overlap[1])
            x_start = min(w - patch_size[1], x_start)
            x_end = x_start + patch_size[1]
            
            # Keep filling the batch
            batch_x[current_batch_size, :, :, :] = im[y_start:y_end, x_start:x_end, :]
            batch_pos.append((y_start, y_end, x_start, x_end))
            patch_count[y_start:y_end, x_start:x_end] += 1
            current_batch_size += 1
            
            if current_batch_size == batch_size or (py == n_patches_y - 1 and px == n_patches_x - 1) :
                # time to predict
                batch_x_normed = patchGenerator.normalize_input(batch_x)
                batch_counts = net.predict(batch_x_normed)
                for b in range(current_batch_size):
                    for i, cnt in enumerate(batch_counts[b].ravel()):
                        counts[i].append(cnt - zeros[i])
                current_batch_size = 0
                
    return counts, batch_pos, patch_count

In [ ]:
im_val = cv2.imread("/home/ubuntu/sealion/data/Train/299.jpg")

In [ ]:
zeros = [0.29107, 0.387059, 0.33593, 0.0]

In [ ]:
#zeros = [0, 0, 0, 0]

In [ ]:
counts, batch_pos, patch_count = predict_counts(im_val, sealion_count_net, trainNoPupsCountsIterator, zeros, overlap=(16, 16))

In [ ]:
for i in range(4):
    print(np.min(counts[i]))

In [ ]:
redundancy = np.sum(patch_count)/patch_count.size
for i in range(4):
    sum_cnt = np.sum(counts[i])
    print(i, ": ", sum_cnt/redundancy)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/home/ubuntu/data/sealion/data/my_correct_counts.csv")

In [ ]:
df[df.train_id == 299]

In [ ]:
sval_ids = set(val_ids)

In [ ]:
df_val = df[df.train_id.isin(sval_ids)]

In [ ]:
df_val.sort_values(by=["adult_females"], ascending=False)

In [ ]:
patch_count.size

In [ ]:
sorted(val_ids)[::-1]